# Lecture 2 - 1

## Soviet planning problem

We are trying to 

The problem that we are trying to solve is 
\begin{align}
\min_{\pi \geq 0} \sum_{(x,y) \in \mathcal{A}} \pi_{xy} c_{xy} \\
\text{s.t. } \nabla^T \pi = n 
\end{align}

In [ ]:
library("gurobi")
library("Matrix")

thePath = getwd()
data = as.matrix(read.csv(paste0(thePath, "/distances.csv"), sep = ",", header = TRUE))  # loads the data
head(data)
tail(data)

In [ ]:
nsources = 68
ndests = 10
dists = matrix(as.numeric(data[1:68, 2:11]), 68, 10)
demand = matrix(as.numeric(data[1:68, 12]))
supply = matrix(as.numeric(data[69, 2:11]))
nonzeros = which(!is.na(dists))

nbNodes = nsources + ndests
nbArcs = length(nonzeros)

rows = (nonzeros - 1)%%nsources + 1
cols = (nonzeros - 1)%/%nsources + 1
c = dists[nonzeros]
arcs = cbind(rows, cols + nsources, c)

n = c(-demand, supply)
nameNodes = c(data[2:(nsources + 1)], dimnames(data)[[2]][2:11])

Construct node-incidence matrix:

In [ ]:
Nabla = sparseMatrix(i = 1:nbArcs, j = arcs[, 1], dims = c(nbArcs, nbNodes), x = -1) + 
    sparseMatrix(i = 1:nbArcs, j = arcs[, 2], dims = c(nbArcs, nbNodes), x = 1)

Now let's solve using Gurobi:
* `A` = $\nabla^T$
* `obj` = $c$
* `sense` = '$=$'
* `rhs` = $n$
* `modelsense` = '$\min$'.

In [ ]:
result = gurobi(list(A = t(Nabla), obj = c, modelsense = "min", rhs = n, sense = "=", 
    start = matrix(0, nbArcs, 1)), params = NULL)
pi = result$x
distance = result$objval

print(distance)